# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
file = pd.read_csv("../output_data/cities.csv")
file.dropna()
file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [28]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [29]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heatmap_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -18.0787379,
                    "lng": 13.8845578
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.07804977010728,
                        "lng": 13.88543002989272
                    },
                    "southwest": {
                        "lat": -18.08074942989272,
                        "lng": 13.88273037010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Kaoko Mopane Lodge & Campsite | Opuwo",
            "opening_hours": {
                "open_no

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBN6ky5wXquB-UwANe5Sh1-2gOuMnMLov4hOQufHaGKuRRrG-lgpFpkWD_XfOnYANTK4SXNiEjgfCH5jF0GwrFB5OasLRrxzbLa8jKCV3eiz5T6bgnsKAIRhHzT2vlDWPyHapj7tdsH2JkGKxWroZrZo8KrL7m8VvcksCwaI1qXMcZ_deOHv9f0-QafDttyAP_67m1A2Iw2WxOZG4mn0GSRw9d0saigrvuz7cKdemynx9S97Xgnk-t_uA5sCMfJZLrPuHzhsz7twGraomkdvMaEM8YHSnkb2NROQEbbV2z29uqtL7LaBYSL8o11QFJlq3lzyjkgBB1taxlLKy76rs9vzd-4pG_BrihVIBmp4ABYCawS1rplWSI3tEdkq88erMeNGN1jGGjUNXEDIY7Xn_Xy70YNt9i27ENJZp5HDFaIrn6l_6LsL96gZ495V-SX",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.7953394,
                    "lng": -50.0371051
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.79399982010728,
                        "lng": -50.03572867010728
                    },
                    "southwest": {
                        "lat": -29.7966

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 14.991011,
                    "lng": 74.04016899999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.99234872989272,
                        "lng": 74.04161817989272
                    },
                    "southwest": {
                        "lat": 14.98964907010728,
                        "lng": 74.03891852010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "The LaLiT Golf & Spa Resort Goa",
            "opening_hours": {
                "open_now":

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.9074985,
                    "lng": 96.47550489999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.90877412989272,
                        "lng": 96.47683007989272
                    },
                    "southwest": {
                        "lat": 22.90607447010728,
                        "lng": 96.47413042010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "King Bridge",
            "opening_hours": {
                "open_now": true
            }

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Kaoko Mopane Lodge & Campsite | Opuwo
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Balneário do Lago Hotel
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Xangri-lá Praia Hotel
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,The LaLiT Golf & Spa Resort Goa
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,King Bridge
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Sepehr Hotel


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))